In [1]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 9.6 MB/s eta 0:00:00:00:0100:01


In [2]:
import fitz

In [ ]:
def extract_text_from_pdf(pdf_path, output_path=None):
    """Extracts text from a PDF and optionally saves it to a text file."""
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)

    if output_txt_path:
        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write(text)
    
    return text

In [97]:
doc = fitz.open("quimica.pdf")

In [9]:
for page in doc:
    print(type(page))

<class 'pymupdf.Page'>
<class 'pymupdf.Page'>
<class 'pymupdf.Page'>
<class 'pymupdf.Page'>
<class 'pymupdf.Page'>
<class 'pymupdf.Page'>
<class 'pymupdf.Page'>
<class 'pymupdf.Page'>
<class 'pymupdf.Page'>
<class 'pymupdf.Page'>


In [ ]:
SUBJECTS = ['QUÍMICA', 'MATEMÁTICAS II', 'FÍSICA']

content = {}

for page in doc[1:]:
    page_dict = {}
    # get content of the page
    text = page.get_text("text")
    
    # clean the content of page
    
    # replacements
    replacements = {'www.emestrada.org' : '',
                    'R  E  S  O  L  U  C  I  Ó  N': 'Resolucion',
                    '\n \n \n' : ''}

    for subs, rep in replacements.items():
        text =  text.replace(subs, rep)
    
    # remove whitespaces and create list of rows
    text = text.strip().replace('  ', ' ').split('\n')

    # extract exam details
    exam_details = text[-1]
    text = text[:-1]

    # extract only exercise statement
    times_a_appeared = 0
    for index, row in enumerate(text):
        if 'a)' in row:
            times_a_appeared += 1
            # it's the second occurrence of a)
            if times_a_appeared == 2:
                text = text[index:]
                break
    
    # clean relative masses
    for index, row in enumerate(text):
        if ('Masas atómicas relativas' in row) or ('Dato:' in row) :
            text = text[:index]
            break
    
    # remove the lines starting with just a number
    text = [i for i in text if len(i) > 1]
    
    # join the lines of the statement
    joined_text = ''
    for row in text:

        if row.startswith('b'):
            joined_text += '\n' + row
        else:
            joined_text += row

    # extract the year and exam
    for subj in SUBJECTS:
        if subj in exam_details:
            index = exam_details.find(subj)
            page_dict['subject'] = subj
            exam = exam_details[index:].split('. ')
            page_dict['year'] = exam[1]
            page_dict['exam'] = exam[2]
            page_dict['exercise'] = exam[3]
    page_dict['page_content'] = joined_text
    page_dict['page_number'] = page.number + 1

    content[f'page_{page.number + 1}'] = page_dict

QUÍMICA. 2024. JUNIO. EJERCICIO C4
QUÍMICA. 2024. RESERVA 1. EJERCICIO C4
QUÍMICA. 2024. RESERVA 2. EJERCICIO B4
QUÍMICA. 2024. RESERVA 2. EJERCICIO C4
QUÍMICA. 2024. RESERVA 3. EJERCICIO B5
QUÍMICA. 2024. RESERVA 3. EJERCICIO C2
QUÍMICA. 2024. RESERVA 4. EJERCICIO B5
QUÍMICA. 2024. RESERVA 4. EJERCICIO C4
QUÍMICA. 2024. JULIO. EJERCICIO C4


In [113]:
content.keys()

dict_keys(['page_2', 'page_3', 'page_4', 'page_5', 'page_6', 'page_7', 'page_8', 'page_9', 'page_10'])

In [129]:
content

{'page_2': {'subject': 'QUÍMICA',
  'year': '2024',
  'exam': 'JUNIO',
  'exercise': 'EJERCICIO C4',
  'page_content': 'a) Ajuste las ecuaciones iónica y molecular por el método del ión-electrón. \nb) Calcule el volumen de Cl obtenido a 0ºC y 1 atm de presión a partir de 30 mL de una disolución 0’5 M de KMnO y 50 mL de una disolución 0’25 M de HCl. ',
  'page_number': 2},
 'page_3': {'subject': 'QUÍMICA',
  'year': '2024',
  'exam': 'RESERVA 1',
  'exercise': 'EJERCICIO C4',
  'page_content': "a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. \nb) Calcule la masa de níquel que podrá oxidarse con 1 mL de ácido nítrico comercial del 70% de riqueza en masa y densidad 1'42 g mL −. ",
  'page_number': 3},
 'page_4': {'subject': 'QUÍMICA',
  'year': '2024',
  'exam': 'RESERVA 2',
  'exercise': 'EJERCICIO B4',
  'page_content': "a) Justifique, escribiendo la semirreacción que tiene lugar en cada uno, cuál es el ánodo y cuál el cátodo. \nb) Calcule el potencial estánd

In [134]:
for exam in content.values():
    print(v['page_content'])
    print('\n' + '*' * 10 + '\n')

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos de Cr(OH) del 90% de riqueza se necesitan para reaccionar completamente con 50 mL de una disolución de KClO 0,55 M? 

**********

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos de Cr(OH) del 90% de riqueza se necesitan para reaccionar completamente con 50 mL de una disolución de KClO 0,55 M? 

**********

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos de Cr(OH) del 90% de riqueza se necesitan para reaccionar completamente con 50 mL de una disolución de KClO 0,55 M? 

**********

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos de Cr(OH) del 90% de riqueza se necesitan para reaccionar completamente con 50 mL de una disolución de KClO 0,55 M? 

**********

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos 

In [135]:
content['page_7']['page_content']

'a) El tiempo necesario para que todo el cobre contenido en 250 mL de una disolución acuosa 0,1 M de iones Cu+ se deposite como cobre metálico, cuando se hace pasar una corriente eléctrica de 1,5 A. \nb) La intensidad de corriente eléctrica que se debe hacer pasar a través de una disolución acuosa de iones Au+ , si se quiere obtener 1 g de oro metálico en 30 minutos. Datos: 96500 C mol −'